Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df = df[['title','label']]

In [7]:
df.head()

,title,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You Fired,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1


In [8]:
df.isnull().sum()

title    558
label      0
dtype: int64

In [11]:
df.dropna(inplace = True)

In [12]:
df.isnull().sum()

title    0
label    0
dtype: int64

In [13]:
X = df['title']
X.head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [14]:
y = df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [15]:
X.shape

(20242,)

In [16]:
import tensorflow as tf
print(tf.__version__)

2.14.0


In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [34]:
##Vocabulary_size = 5000
voc_size = 5000

### One hot representation

In [19]:
X[1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
X.head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [29]:
X.iloc[0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [30]:
### Data preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0,len(X)):
  review = re.sub('[^a-zA-Z]',' ',X.iloc[i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [31]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

In [32]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [36]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
print(onehot_repr)

[[2383, 2351, 813, 3168, 1620, 74, 3977, 3998, 3219, 3673], [2369, 1833, 3974, 335, 1079, 775, 2941], [4308, 2917, 2369, 2015], [3940, 3020, 2991, 2822, 4393, 1106], [673, 1079, 2967, 589, 1290, 224, 1079, 4628, 3401, 4475], [2082, 3718, 4450, 4856, 2069, 439, 4718, 1257, 2516, 2161, 1840, 3889, 1750, 4551, 2941], [965, 965, 4808, 569, 2525, 189, 3489, 1768, 2961, 3717, 2330, 1004], [76, 469, 786, 2952, 1359, 3488, 2663, 4668, 1261, 2230, 3465], [2119, 3273, 4360, 2438, 439, 419, 1044, 4751, 4369, 1698, 1261, 2230, 3465], [145, 1451, 2562, 3955, 3892, 3719, 439, 276, 1261, 2230, 3465], [4669, 118, 1850, 422, 3084, 3812, 1753, 3996, 439, 4804], [2233, 2367, 2799, 586, 4711, 2012, 1970, 2098], [3989, 1184, 4953, 3568, 3546, 887, 1651, 4740, 2724, 2488, 2390], [2822, 3542, 1620, 3812, 439, 3892], [2453, 894, 531, 1324, 4341, 732, 2986, 1825, 3951], [2936, 3259, 2963, 1180, 3362, 4015, 3873, 1261, 2230, 3465], [557, 3753, 889, 814, 3274, 1261, 2230, 3465], [4434, 2023, 2841, 4504, 393, 414

In [37]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [38]:
onehot_repr[1]

[2369, 1833, 3974, 335, 1079, 775, 2941]

### Embedding Representation

In [41]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding = 'post',maxlen = sent_length)
print(embedded_docs)

[[2383 2351  813 ...    0    0    0]
 [2369 1833 3974 ...    0    0    0]
 [4308 2917 2369 ...    0    0    0]
 ...
 [ 263 2494 4417 ...    0    0    0]
 [4632 3892 3647 ...    0    0    0]
 [2725 2321 3722 ...    0    0    0]]


In [42]:
embedded_docs[1]

array([2369, 1833, 3974,  335, 1079,  775, 2941,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [43]:
## Creating model

embedding_vector_features = 40 ##feature representation

model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics =['accuracy'])


In [44]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
len(embedded_docs)

20242

In [47]:
y.shape

(20242,)

In [48]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [49]:
X_final.shape,y_final.shape

((20242, 20), (20242,))

In [107]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size =0.33,random_state =42)

### Model Training

In [52]:
### FInally Training
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs =10,batch_size = 64)

Epoch 1/10
212/212 [==============================] - 22s 70ms/step - loss: 0.2991 - accuracy: 0.8665 - val_loss: 0.1986 - val_accuracy: 0.9223
Epoch 2/10
212/212 [==============================] - 3s 16ms/step - loss: 0.1334 - accuracy: 0.9513 - val_loss: 0.1932 - val_accuracy: 0.9219
Epoch 3/10
212/212 [==============================] - 4s 17ms/step - loss: 0.0967 - accuracy: 0.9660 - val_loss: 0.2095 - val_accuracy: 0.9174
Epoch 4/10
212/212 [==============================] - 2s 11ms/step - loss: 0.0700 - accuracy: 0.9757 - val_loss: 0.2958 - val_accuracy: 0.9127
Epoch 5/10
212/212 [==============================] - 2s 10ms/step - loss: 0.0547 - accuracy: 0.9821 - val_loss: 0.2779 - val_accuracy: 0.9184
Epoch 6/10
212/212 [==============================] - 2s 9ms/step - loss: 0.0428 - accuracy: 0.9857 - val_loss: 0.3611 - val_accuracy: 0.9105
Epoch 7/10
212/212 [==============================] - 2s 8ms/step - loss: 0.0351 - accuracy: 0.9892 - val_loss: 0.3389 - val_accuracy: 0.9154


### Adding Dropout

In [64]:
from tensorflow.keras.layers import Dropout

In [154]:
## Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])

In [155]:
### FInally Training
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs =10,batch_size = 64)

Epoch 1/10
212/212 [==============================] - 12s 47ms/step - loss: 0.3017 - accuracy: 0.8591 - val_loss: 0.2003 - val_accuracy: 0.9175
Epoch 2/10
212/212 [==============================] - 3s 13ms/step - loss: 0.1427 - accuracy: 0.9451 - val_loss: 0.2032 - val_accuracy: 0.9268
Epoch 3/10
212/212 [==============================] - 2s 9ms/step - loss: 0.1077 - accuracy: 0.9609 - val_loss: 0.2091 - val_accuracy: 0.9235
Epoch 4/10
212/212 [==============================] - 3s 12ms/step - loss: 0.0844 - accuracy: 0.9697 - val_loss: 0.2250 - val_accuracy: 0.9205
Epoch 5/10
212/212 [==============================] - 2s 10ms/step - loss: 0.0708 - accuracy: 0.9748 - val_loss: 0.2962 - val_accuracy: 0.9186
Epoch 6/10
212/212 [==============================] - 2s 8ms/step - loss: 0.0532 - accuracy: 0.9813 - val_loss: 0.3379 - val_accuracy: 0.9135
Epoch 7/10
212/212 [==============================] - 2s 8ms/step - loss: 0.0434 - accuracy: 0.9840 - val_loss: 0.3732 - val_accuracy: 0.9136
E

# Performance Metrics and Accuracy

In [156]:
y_pred = model.predict(X_test)

209/209 [==============================] - 1s 3ms/step


In [157]:
y_pred

array([[5.8405090e-02],
       [1.6998030e-05],
       [5.6665631e-05],
       ...,
       [9.9967802e-01],
       [9.9669707e-01],
       [9.9934238e-01]], dtype=float32)

In [158]:
y_pred = np.where(y_pred > 0.5,1,0) ## AUC ROC curve

In [159]:
from sklearn.metrics import confusion_matrix

In [160]:
confusion_matrix(y_test,y_pred)

array([[3087,  323],
       [ 228, 3042]])

In [161]:
from sklearn.metrics import accuracy_score,classification_report

In [162]:
accuracy_score(y_test,y_pred)

0.9175149700598803

In [163]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3410
           1       0.90      0.93      0.92      3270

    accuracy                           0.92      6680
   macro avg       0.92      0.92      0.92      6680
weighted avg       0.92      0.92      0.92      6680

